In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from apscheduler.schedulers.background import BackgroundScheduler

from contextlib import contextmanager

from rich.console import Console
from rich.table import Table
from rich.live import Live

from getpass import getpass

from datetime import datetime, timezone
import re
import random
import time

console = Console()

ADVENTURES = 'adventures'
RAIDS = 'raids'
logs = {
    ADVENTURES: '',
    RAIDS: '',
}

In [3]:
''' scheduler init. '''

def calc_new_interval_between(x, y):
    return random.uniform(x, y)

def cleanup_scheduler(scheduler):
    if scheduler.running:
        print("Shutting down scheduler...")
        scheduler.shutdown(wait=True)
    else:
        print("Scheduler already stopped.")

# Context manager for BackgroundScheduler
@contextmanager
def managed_scheduler(*args, **kwargs):
    scheduler = BackgroundScheduler(*args, **kwargs)
    scheduler.start()
    try:
        yield scheduler
    finally:
        cleanup_scheduler(scheduler)

In [4]:
#             roman,      roman,   hun,       gaul,     egyptian,     teuton
usernames = ['robot723', 'athos', 'porthos', 'aramis', 'd.artagnan', 'richelieu']
passwords = ['Robot6210!', 'Athos2024!', 'Porthos2024!', 'Aramis2024!', 'dArtagnan2024!', 'Richelieu2024!']

In [5]:
''' Open Travian International 2 server and login if not already '''

def attempt_login(site, username, password, driver):
    driver.get(site)

    try:
        input_username = driver.find_element(By.XPATH, '//*[@id="loginForm"]/tbody/tr[1]/td[2]/input')
        input_pwd = driver.find_element(By.XPATH, '//*[@id="loginForm"]/tbody/tr[2]/td[2]/input')
        button_login = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"][value="Login"].textButtonV1.green')

        input_username.send_keys(username)
        input_pwd.send_keys(password)
        button_login.click()
    except NoSuchElementException:
        print("User already signed in!")

In [6]:
''' Send hero on adventure '''

def attempt_to_start_adventure(driver, scheduler):
    button_hero_status = driver.find_element(By.XPATH, '//*[@id="topBarHero"]/div/a/i')

    # dynamic ID, using href to reference
    button_adventures = driver.find_element(By.XPATH, '//a[contains(@href, "/hero/adventures")]')
    try:
        num_adventures = button_adventures.find_element(By.XPATH, "./div")
    except NoSuchElementException:
        num_adventures = None

    if num_adventures and 'heroHome' in button_hero_status.get_attribute('class') and int(num_adventures.text) > 0:
        button_adventures.click()

        button_start_first_adventure = driver.find_element(By.XPATH, '//*[@id="heroAdventure"]/table/tbody/tr[1]/td[5]/button')
        button_start_first_adventure.click()

        button_continue = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="heroAdventure"]/div/button')))
        button_continue.click()

        logs[ADVENTURES] = "Successfully sent out hero on adventure!"
    elif num_adventures is None:
        logs[ADVENTURES] = "No adventures found."
    elif 'heroRunning' in button_hero_status.get_attribute('class'):
        logs[ADVENTURES] = "Hero is already on an adventure!"
    else:
        logs[ADVENTURES] = "Error attempting to start adventure."
    
    scheduler.add_job(attempt_to_start_adventure, 'interval', seconds=calc_new_interval_between(307, 902), id=ADVENTURES, args=[driver, scheduler], replace_existing=True)

In [7]:
''' Upgrade lowest level field '''

def attempt_to_upgrade_field(driver):
    button_resource_fields_page = driver.find_element(By.XPATH, '//*[@id="navigation"]/a[1]')
    button_resource_fields_page.click()

    resource_field_container = driver.find_element(By.XPATH, '//*[@id="resourceFieldContainer"]')
    resource_field_links = resource_field_container.find_elements(By.TAG_NAME, 'a')

    # add all fields that we can upgrade to new list
    upgradable_fields = []
    for link in resource_field_links:
        if 'good' in link.get_attribute('class'):
            upgradable_fields.append(link)

    # find the lowest level field
    lowest_level_field = None
    lowest_level = 999
    for field in upgradable_fields:
        # do not upgrade fields that are already being upgraded
        if 'underConstruction' in field.get_attribute('class'): continue

        div_curr_level = field.find_element(By.XPATH, './div')
        curr_level = int(div_curr_level.text.strip() or 0)
        if curr_level < lowest_level:
            lowest_level_field = field
            lowest_level = curr_level

    if lowest_level_field:
        lowest_level_field.click()

        button_upgrade = driver.find_element(By.CSS_SELECTOR, '.textButtonV1.green.build')
        button_upgrade.click()
    else:
        print("No upgradable fields were found!")

    #scheduler.add_job(attempt_to_upgrade_field, 'interval', seconds=calc_new_interval_between(58, 307), replace_existing=True)

In [8]:
def activate_farm_list_raids_for(id, driver):
    name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f'{id}/div/div[1]/div[2]/div[1]'))).text

    table = driver.find_element(By.XPATH, f'{id}/div/div[2]/table')
    table_body = table.find_element(By.XPATH, './tbody')
    table_footer = table.find_element(By.XPATH, './tfoot')
    table_rows = table_body.find_elements(By.TAG_NAME, 'tr')

    for row in table_rows:
        try:
            checkbox = row.find_element(By.XPATH, './td[1]/label/input')
        except NoSuchElementException:
            continue

        last_raid_state_container = row.find_element(By.XPATH, './td[7]')
        try:
            last_raid_state = last_raid_state_container.find_element(By.TAG_NAME, 'i')
        except NoSuchElementException:
            last_raid_state = None
        
        curr_state_container = row.find_element(By.XPATH, './td[2]')
        try:
            curr_state = curr_state_container.find_element(By.TAG_NAME, 'i')
        except NoSuchElementException:
            curr_state = None

        num_troops_container = row.find_element(By.XPATH, './td[6]')
        num_troops = int(num_troops_container.find_element(By.XPATH, './div[1]/span[1]/span[1]').text)

        try:
            filthy_nominator   = table_footer.find_element(By.XPATH, './tr[1]/td[2]/div/div/span/span/span[1]').text
            filthy_denominator = table_footer.find_element(By.XPATH, './tr[1]/td[2]/div/div/span/span/span[2]').text
            nominator   = int(re.sub(r'[^\d]', '', filthy_nominator))
            denominator = int(re.sub(r'[^\d]', '', filthy_denominator))
        except NoSuchElementException:
            nominator = denominator = None

        if (not curr_state or 'attack_small' not in curr_state.get_attribute('class')) and \
        (not last_raid_state or 'attack_won_withoutLosses_small' in last_raid_state.get_attribute('class')) and \
        ((not nominator and not denominator) or nominator + num_troops <= denominator):
            checkbox.click()

    # start raids
    button_start_raids = driver.find_element(By.XPATH, f'{id}/div/div[1]/button')
    button_start_raids.click()

    logs[RAIDS] = f'Finished raid logic for {name}'

In [9]:
def send_troops_to_farm(driver, scheduler):
    if driver.current_url != 'https://ts2.x1.international.travian.com/build.php?id=39&gid=16&tt=99':
        button_buildings_page = driver.find_element(By.XPATH, '//*[@id="navigation"]/a[2]')
        button_buildings_page.click()

        button_rally_point = driver.find_element(By.XPATH, '//*[@id="villageContent"]/div[21]/a')
        button_rally_point.click()

        button_farm_list = driver.find_element(By.XPATH, '//a[contains(@href, "/build.php?id=39&gid=16&tt=99")]')
        button_farm_list.click()

    oases    = '//*[@id="rallyPointFarmList"]/div[2]/div[2]'
    activate_farm_list_raids_for(oases, driver)

    villages = '//*[@id="rallyPointFarmList"]/div[2]/div[3]'
    activate_farm_list_raids_for(villages, driver)

    scheduler.add_job(send_troops_to_farm, 'interval', seconds=calc_new_interval_between(183, 302), id=RAIDS, args=[driver, scheduler], replace_existing=True)

    logs[RAIDS] = "Finished raid attempt."

In [10]:
def generate_job_scheduler_table_from(scheduler):
    table = Table(title="Scheduled Jobs")

    table.add_column("Job ID", style="cyan", no_wrap=True)
    table.add_column("Next Run At", style="magenta")
    table.add_column("Countdown", style="green")
    table.add_column("Log", style="blue")

    now = datetime.now(timezone.utc)
    for job in scheduler.get_jobs():
        next_run = job.next_run_time
        countdown = (next_run - now).total_seconds()
        table.add_row(job.id, str(next_run), str(int(countdown)), logs[job.id])
    
    return table

In [11]:
''' web driver init. '''
def init_webdriver():
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

''' init. and login to server '''
input_site = 'https://ts2.x1.international.travian.com/dorf1.php'
#input_site = input("Site:") # Commented out while testing
input_username = input("Username:")
input_password = getpass("Password:")

driver = init_webdriver()
attempt_login(input_site, input_username, input_password, driver)

''' begin scheduler tasks '''
with managed_scheduler() as scheduler:
    scheduler.add_job(attempt_to_start_adventure, 'interval', seconds=1, id=ADVENTURES, args=[driver, scheduler])
    scheduler.add_job(send_troops_to_farm, 'interval', seconds=20, id=RAIDS, args=[driver, scheduler])

    with Live(generate_job_scheduler_table_from(scheduler), refresh_per_second=1, console=console) as live:
        try:
            while True:
                live.update(generate_job_scheduler_table_from(scheduler))
                time.sleep(1)
        except (KeyboardInterrupt, SystemExit):
            pass

Output()

Job "attempt_to_start_adventure (trigger: interval[0:00:01], next run at: 2024-03-13 18:07:05 MDT)" raised an 
exception
Traceback (most recent call last):
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/executors/base.py", 
line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_48106/1547744773.py", line 30, in 
attempt_to_start_adventure
    scheduler.add_job(attempt_to_start_adventure, 'interval', seconds=calc_new_interval_between(307, 902), 
id=ADVENTURES, replace_existing=True)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/schedulers/base.py", 
line 448, in add_job
    job = Job(self, **job_kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/job.py", line 
49, in __init__
    self._modify(id=id or uuid4().hex, **kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/job.py", line 
180, in _modify
    check_callable_args(func, args, kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/util.py", line 
372, in check_callable_args
    raise ValueError('The following arguments have not been supplied: %s' %
ValueError: The following arguments have not been supplied: driver, scheduler

Job "attempt_to_start_adventure (trigger: interval[0:00:01], next run at: 2024-03-13 18:07:06 MDT)" raised an 
exception
Traceback (most recent call last):
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/executors/base.py", 
line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_48106/1547744773.py", line 30, in 
attempt_to_start_adventure
    scheduler.add_job(attempt_to_start_adventure, 'interval', seconds=calc_new_interval_between(307, 902), 
id=ADVENTURES, replace_existing=True)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/schedulers/base.py", 
line 448, in add_job
    job = Job(self, **job_kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/job.py", line 
49, in __init__
    self._modify(id=id or uuid4().hex, **kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/job.py", line 
180, in _modify
    check_callable_args(func, args, kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/util.py", line 
372, in check_callable_args
    raise ValueError('The following arguments have not been supplied: %s' %
ValueError: The following arguments have not been supplied: driver, scheduler

Job "attempt_to_start_adventure (trigger: interval[0:00:01], next run at: 2024-03-13 18:07:07 MDT)" raised an 
exception
Traceback (most recent call last):
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/executors/base.py", 
line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_48106/1547744773.py", line 30, in 
attempt_to_start_adventure
    scheduler.add_job(attempt_to_start_adventure, 'interval', seconds=calc_new_interval_between(307, 902), 
id=ADVENTURES, replace_existing=True)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/schedulers/base.py", 
line 448, in add_job
    job = Job(self, **job_kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/job.py", line 
49, in __init__
    self._modify(id=id or uuid4().hex, **kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/job.py", line 
180, in _modify
    check_callable_args(func, args, kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/util.py", line 
372, in check_callable_args
    raise ValueError('The following arguments have not been supplied: %s' %
ValueError: The following arguments have not been supplied: driver, scheduler

Job "attempt_to_start_adventure (trigger: interval[0:00:01], next run at: 2024-03-13 18:07:08 MDT)" raised an 
exception
Traceback (most recent call last):
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/executors/base.py", 
line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_48106/1547744773.py", line 30, in 
attempt_to_start_adventure
    scheduler.add_job(attempt_to_start_adventure, 'interval', seconds=calc_new_interval_between(307, 902), 
id=ADVENTURES, replace_existing=True)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/schedulers/base.py", 
line 448, in add_job
    job = Job(self, **job_kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/job.py", line 
49, in __init__
    self._modify(id=id or uuid4().hex, **kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/job.py", line 
180, in _modify
    check_callable_args(func, args, kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/util.py", line 
372, in check_callable_args
    raise ValueError('The following arguments have not been supplied: %s' %
ValueError: The following arguments have not been supplied: driver, scheduler

Job "attempt_to_start_adventure (trigger: interval[0:00:01], next run at: 2024-03-13 18:07:09 MDT)" raised an 
exception
Traceback (most recent call last):
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/executors/base.py", 
line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_48106/1547744773.py", line 30, in 
attempt_to_start_adventure
    scheduler.add_job(attempt_to_start_adventure, 'interval', seconds=calc_new_interval_between(307, 902), 
id=ADVENTURES, replace_existing=True)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/schedulers/base.py", 
line 448, in add_job
    job = Job(self, **job_kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/job.py", line 
49, in __init__
    self._modify(id=id or uuid4().hex, **kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/job.py", line 
180, in _modify
    check_callable_args(func, args, kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/util.py", line 
372, in check_callable_args
    raise ValueError('The following arguments have not been supplied: %s' %
ValueError: The following arguments have not been supplied: driver, scheduler

Job "attempt_to_start_adventure (trigger: interval[0:00:01], next run at: 2024-03-13 18:07:10 MDT)" raised an 
exception
Traceback (most recent call last):
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/executors/base.py", 
line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_48106/1547744773.py", line 30, in 
attempt_to_start_adventure
    scheduler.add_job(attempt_to_start_adventure, 'interval', seconds=calc_new_interval_between(307, 902), 
id=ADVENTURES, replace_existing=True)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/schedulers/base.py", 
line 448, in add_job
    job = Job(self, **job_kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/job.py", line 
49, in __init__
    self._modify(id=id or uuid4().hex, **kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/job.py", line 
180, in _modify
    check_callable_args(func, args, kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/util.py", line 
372, in check_callable_args
    raise ValueError('The following arguments have not been supplied: %s' %
ValueError: The following arguments have not been supplied: driver, scheduler

Job "attempt_to_start_adventure (trigger: interval[0:00:01], next run at: 2024-03-13 18:07:11 MDT)" raised an 
exception
Traceback (most recent call last):
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/executors/base.py", 
line 125, in run_job
    retval = job.func(*job.args, **job.kwargs)
  File "/var/folders/tf/s_38qxxn53jg20v0_kpkc6580000gp/T/ipykernel_48106/1547744773.py", line 30, in 
attempt_to_start_adventure
    scheduler.add_job(attempt_to_start_adventure, 'interval', seconds=calc_new_interval_between(307, 902), 
id=ADVENTURES, replace_existing=True)
  File 
"/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/schedulers/base.py", 
line 448, in add_job
    job = Job(self, **job_kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/job.py", line 
49, in __init__
    self._modify(id=id or uuid4().hex, **kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/job.py", line 
180, in _modify
    check_callable_args(func, args, kwargs)
  File "/Users/danielgarcia/SideProjects/TravianAuto/.venv/lib/python3.9/site-packages/apscheduler/util.py", line 
372, in check_callable_args
    raise ValueError('The following arguments have not been supplied: %s' %
ValueError: The following arguments have not been supplied: driver, scheduler

Shutting down scheduler...
